## mclogit installation

We first install mclogit. Its not included in the env.yaml because it can only be installed from within R. Only run the cell below once in the environment to install it.

In [1]:
# only run this cell once in the R version of the conda environment
install.packages("mclogit")

also installing the dependencies ‘lattice’, ‘MASS’, ‘data.table’, ‘yaml’, ‘Matrix’, ‘memisc’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


Now we still need to load the packages, same as import of python modules. This needs to be done each time for the notebook.

We also need to import some functions for the analysis of the statistical models.

In [4]:
library("mclogit")
source("mclogit-effects.r")

## Importing the dataframe to R

In [6]:
# we can directly read the stored pandas dataframe to R
df_wt <- read.csv("../data/results/dataframe.csv")

In [7]:
# inspect the R dataframe
head(df_wt, n=5)

X,rlnCoordinateX,rlnCoordinateY,rlnCoordinateZ,rlnAngleRot,rlnAngleTilt,rlnAnglePsi,rlnImageName,rlnCtfImage,rlnRandomSubset,...,state,elongation,trapccdc,state_full,trailing_id,leading_id,tr_state_full,ld_state_full,tr_elongation,ld_elongation
0,872.7986,529.7619,286.2439,11.28056,82.88946,181.65190,/data2/mgemmer/WARPM/HEKFWT/000_Subtomograms/001_TM_bin4/tomo1.mrc/tomo1.mrc_Particles_TM_0000082_6.90A.mrc,/data2/mgemmer/WARPM/HEKFWT/000_Subtomograms/001_TM_bin4/tomo1.mrc/tomo1.mrc_Particles_TM_0000082_ctf_6.90A.mrc,1,...,NCLN,Dec,ap,NCLNCCDC47,-1,-1,non,non,non,non
1,401.9046,214.0999,138.9087,-61.55985,136.25310,-135.14440,/data2/mgemmer/WARPM/HEKFWT/000_Subtomograms/001_TM_bin4/tomo1.mrc/tomo1.mrc_Particles_TM_0000074_6.90A.mrc,/data2/mgemmer/WARPM/HEKFWT/000_Subtomograms/001_TM_bin4/tomo1.mrc/tomo1.mrc_Particles_TM_0000074_ctf_6.90A.mrc,1,...,OST,Dec,Unk,OST,43744,-1,OST,non,Rot1,non
2,824.8273,699.2300,245.7589,-54.52035,85.32531,102.23010,/data2/mgemmer/WARPM/HEKFWT/000_Subtomograms/001_TM_bin4/tomo1.mrc/tomo1.mrc_Particles_TM_0000066_6.90A.mrc,/data2/mgemmer/WARPM/HEKFWT/000_Subtomograms/001_TM_bin4/tomo1.mrc/tomo1.mrc_Particles_TM_0000066_ctf_6.90A.mrc,1,...,OST,Dec,Unk,OST,22540,-1,TRAP,non,Pre+,non
3,854.6401,663.1552,180.3849,135.56340,108.82990,170.21040,/data2/mgemmer/WARPM/HEKFWT/000_Subtomograms/001_TM_bin4/tomo1.mrc/tomo1.mrc_Particles_TM_0000131_6.90A.mrc,/data2/mgemmer/WARPM/HEKFWT/000_Subtomograms/001_TM_bin4/tomo1.mrc/tomo1.mrc_Particles_TM_0000131_ctf_6.90A.mrc,2,...,OST,Dec,Unk,OST,-1,-1,non,non,non,non
4,365.4104,783.4767,171.0251,-170.52960,96.45240,78.46428,/data2/mgemmer/WARPM/HEKFWT/000_Subtomograms/001_TM_bin4/tomo1.mrc/tomo1.mrc_Particles_TM_0000084_6.90A.mrc,/data2/mgemmer/WARPM/HEKFWT/000_Subtomograms/001_TM_bin4/tomo1.mrc/tomo1.mrc_Particles_TM_0000084_ctf_6.90A.mrc,1,...,TRAP,Dec,Unk,TRAP,93748,77732,Unk,Sol,Rot2,Rot2


In [8]:
# add some annotation => one for whether the ribosome has any polysome connections
# this way we can test the probability of classes being present in a polysome chain
df_wt$in_chain <- ((df_wt$trailing_id != -1) | (df_wt$leading_id != -1)) # has trailing or leading connection
df_wt$in_chain <- factor(df_wt$in_chain)

## Fitting a multinomial logit model with mixed effects

For the random parameter we first set (~1|date) meaning a random intercept per date, but fixed slope.

In [28]:
# create temporary dataframe with only ribosomes assigned to an elongation cycle intermediate
temp <- df_wt[ df_wt$elongation %in% c('Dec', 'Post', 'Pre', 'Pre+', 'Rot1', 'Rot1+', 
         'Rot2', 'RotIdle', 'Translocation', 'UnRotIdle'), , drop=FALSE ]
temp$elongation <- factor(temp$elongation)

model_elongation_state_in_chain_1a <- mblogit(in_chain ~ elongation, data = temp, random = ~1|date,
                       control=mmclogit.control(epsilon = 1e-08, maxit = 100))
summary(model_elongation_state_in_chain_1a)


Iteration 1 - deviance = 172531.6 - criterion = 1.861356
Iteration 2 - deviance = 172286.4 - criterion = 0.02638476
Iteration 3 - deviance = 172282 - criterion = 0.001012971
Iteration 4 - deviance = 172282 - criterion = 3.173725e-06
Iteration 5 - deviance = 172282 - criterion = 2.818981e-11
converged



Call:
mblogit(formula = in_chain ~ elongation, data = temp, random = ~1 | 
    date, control = mmclogit.control(epsilon = 1e-08, maxit = 100))

Equation for TRUE vs FALSE:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -0.01354    0.07759  -0.174 0.861522    
elongationPost          -0.10493    0.02115  -4.961 7.01e-07 ***
elongationPre            0.18662    0.03472   5.375 7.64e-08 ***
elongationPre+           0.30819    0.01593  19.347  < 2e-16 ***
elongationRot1           0.38052    0.03083  12.342  < 2e-16 ***
elongationRot1+          0.80278    0.03614  22.212  < 2e-16 ***
elongationRot2           0.33712    0.01883  17.904  < 2e-16 ***
elongationRotIdle       -2.83749    0.06312 -44.956  < 2e-16 ***
elongationTranslocation -0.10007    0.02903  -3.447 0.000567 ***
elongationUnRotIdle     -1.46086    0.03109 -46.993  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Co-)Variances:
Grouping level: date 
      

In [13]:
fit.eff <- Effect.mblogit(model_elongation_state_in_chain_1a, 'elongation')
fit.eff$predictors  # these are the row names
data.frame(fit.eff$prob, fit.eff$lower.prob, fit.eff$upper.prob)  # of this table

[1] "Dec"           "Post"          "Pre"           "Pre+"         
 [5] "Rot1"          "Rot1+"         "Rot2"          "RotIdle"      
 [9] "Translocation" "UnRotIdle"

prob.FALSE,prob.TRUE,L.prob.FALSE,L.prob.TRUE,U.prob.FALSE,U.prob.TRUE
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.5033837,0.49661625,0.4654189,0.45869034,0.5413097,0.53458114
0.5295805,0.47041949,0.4911488,0.43233532,0.5676647,0.50885124
0.4568367,0.54316335,0.4167253,0.50248428,0.4975157,0.58327475
0.4268644,0.57313556,0.3902731,0.53572584,0.4642742,0.60972686
0.4092704,0.59072956,0.3712328,0.55157407,0.4484259,0.62876720
0.3123298,0.68767019,0.2782021,0.65137804,0.3486220,0.72179787
0.4198035,0.58019655,0.3831416,0.54262771,0.4573723,0.61685838
0.9453716,0.05462841,0.9344957,0.04547044,0.9545296,0.06550426
0.5283699,0.47163013,0.4886963,0.43231173,0.5676883,0.51130372


In [ ]:
# we can right the results to a csv file as such:
prediction <- data.frame(fit.eff$prob, fit.eff$lower.prob, fit.eff$upper.prob)
write.csv(prediction, "../data/results/mblogit_elongation-state-in-chain_rand-intercept.csv", row.names=TRUE)

We can attempt to make the random effects more complex by making both the intercept and the slope random (~elongation|date). But for this we get convergence issues due to the large number of data points and logistic equations.

In [29]:
# create temporary dataframe with only ribosomes assigned to an elongation cycle intermediate
temp <- df_wt[ df_wt$elongation %in% c('Dec', 'Post', 'Pre', 'Pre+', 'Rot1', 'Rot1+', 
         'Rot2', 'RotIdle', 'Translocation', 'UnRotIdle'), , drop=FALSE ]
temp$elongation <- factor(temp$elongation)

model_elongation_state_in_chain_1b <- mblogit(in_chain ~ elongation, data = temp, 
                                              random = list(~elongation|date),
                       control=mmclogit.control(epsilon = 1e-08, maxit = 100))
summary(model_elongation_state_in_chain_1b)

Warning message:
“Inner iterations did not coverge - nlminb message: false convergence (8)”


Iteration 1 - deviance = 172491 - criterion = 1.847864

Warning message:
“Inner iterations did not coverge - nlminb message: false convergence (8)”


Iteration 2 - deviance = 172236.2 - criterion = 0.0272795

Warning message:
“Inner iterations did not coverge - nlminb message: false convergence (8)”

...

Warning message:
“Inner iterations did not coverge - nlminb message: false convergence (8)”


Iteration 4 - deviance = 172230.9 - criterion = 3.526108e-06

Warning message:
“Inner iterations did not coverge - nlminb message: false convergence (8)”


Iteration 5 - deviance = 172230.9 - criterion = 3.460444e-11
converged



Call:
mblogit(formula = in_chain ~ elongation, data = temp, random = list(~elongation | 
    date), control = mmclogit.control(epsilon = 1e-08, maxit = 100))

Equation for TRUE vs FALSE:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)              0.01124    0.06393   0.176 0.860398    
elongationPost          -0.11312    0.03903  -2.898 0.003756 ** 
elongationPre            0.16721    0.05034   3.321 0.000896 ***
elongationPre+           0.26079    0.05564   4.687 2.77e-06 ***
elongationRot1           0.36188    0.05365   6.746 1.52e-11 ***
elongationRot1+          0.76278    0.06040  12.630  < 2e-16 ***
elongationRot2           0.29520    0.05213   5.663 1.49e-08 ***
elongationRotIdle       -2.61256    0.18454 -14.157  < 2e-16 ***
elongationTranslocation -0.12391    0.04305  -2.878 0.003998 ** 
elongationUnRotIdle     -1.34075    0.11036 -12.149  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Co-)Variances:
Grouping leve

Instead we can also attempt to add a random intercept for both the date and tomogram id, by either putting `list(~1|date, ~1|tomogram)` or `~1|date/tomogram`. The latter works as these random effects are nested, i.e. a tomogram was always collected during a specific collection sessions (the date represents collection session).

In [11]:
# create temporary dataframe with only ribosomes assigned to an elongation cycle intermediate
temp <- df_wt[ df_wt$elongation %in% c('Dec', 'Post', 'Pre', 'Pre+', 'Rot1', 'Rot1+', 
         'Rot2', 'RotIdle', 'Translocation', 'UnRotIdle'), , drop=FALSE ]
temp$elongation <- factor(temp$elongation)

model_elongation_state_in_chain_1b <- mblogit(in_chain ~ elongation, data = temp, random = list(~1|date, ~1|tomogram),
                       control=mmclogit.control(epsilon = 1e-08, maxit = 100))
summary(model_elongation_state_in_chain_1b)


Iteration 1 - deviance = 170310.2 - criterion = 1.575212
Iteration 2 - deviance = 170171.3 - criterion = 0.02305999
Iteration 3 - deviance = 170173 - criterion = 0.001032284
Iteration 4 - deviance = 170172 - criterion = 3.98744e-06
Iteration 5 - deviance = 170172 - criterion = 5.753329e-11
converged



Call:
mblogit(formula = in_chain ~ elongation, data = temp, random = list(~1 | 
    date, ~1 | tomogram), control = mmclogit.control(epsilon = 1e-08, 
    maxit = 100))

Equation for TRUE vs FALSE:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -0.04566    0.08927  -0.512  0.60897    
elongationPost          -0.09708    0.02148  -4.520 6.18e-06 ***
elongationPre            0.18891    0.03523   5.362 8.24e-08 ***
elongationPre+           0.31163    0.01616  19.280  < 2e-16 ***
elongationRot1           0.37971    0.03131  12.128  < 2e-16 ***
elongationRot1+          0.81705    0.03661  22.319  < 2e-16 ***
elongationRot2           0.34359    0.01915  17.945  < 2e-16 ***
elongationRotIdle       -2.83913    0.06356 -44.672  < 2e-16 ***
elongationTranslocation -0.09209    0.02945  -3.127  0.00176 ** 
elongationUnRotIdle     -1.45999    0.03161 -46.188  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Co-)Variances:
Gr

The variance-covariance matrix of the random effects (although now its just a single value), shows how much the output levels vary with the random variable. In this case we have one output level (true or false, but this can be condensed to a single value), and we see how much this varies with the random effect we included. In case of a matrix we dont want to see high values on the off-diagonals, higher values on the diagonals however just mean that the predictor varies with the date or tomogram number. This can also be a bad thing but depends on the variable and its abundance. In this case we see there is more variation per tomogram than there is per date. This is unsurprising as each date has many observations (approx. 20000 per date) while each tomogram has relatively little observations (approx. 100 per tomogram). The latter will inherently be more noisy. Its a good confirmation that there is no strange variance in the data though. 